In [ ]:
#: the usual imports
import babypandas as bpd
import numpy as np
import sys
sys.path.append('./data')

%matplotlib inline
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')

plt.style.use('fivethirtyeight')

# Lecture 19

### Causality and Bootstrapping

## Last Time: Permutation Testing (A/B Testing)
- Given two samples, are they drawn from the same population?
- Answers questions like:
    - "Do smoking moms and nonsmoking moms have babies that weigh the same?"
    - "Did the Colts' footballs and Patriots' footballs have the same amount of pressure drop?"
    - Are *these things* like *those things*?
 - Can also be used to establish causality, in a randomized controlled trial.
     - Randomizing the treatment and control group avoids confounding factors.

# Causality Example : Chronic Back Pain

## Treatment with Botulinum Toxin A (Botox)

- "Botulinum toxin is the most poisonous substance known" [Wikipedia](https://en.wikipedia.org/wiki/Botulinum_toxin).
- Used for treating muscle disorders, migraines, and for cosmetic purposes.
- A randomized controlled study examined the use of Botox in the treatment of lower back pain.
    - 31 patients with pain were randomly assigned to control and treatment groups.
    - Control group received placebo (saline).
    - Treatment group received Botox.
    - After eight weeks, the number of people who experienced relief was counted.

## The data

In [ ]:
#: 1 means "experienced relief", 0 means "no relief"
back = bpd.read_csv('./data/bta.csv')
back

## The results

In [ ]:
#: proportion experiencing relief
back.groupby('Group').mean()

- It looks like there was a clear benefit to treatment.
- But can this just be due to chance?

## An A/B test

Group A = Control, Group B = Treatment

- **Null hypothesis**: Results for both groups come from the same distribution.
- **Alternative hypothesis**: More people in the treatment group experience relief.

**Test statistic**: difference in proportion experiencing relief

In [ ]:
#: proportion experiencing relief
relief = back.groupby('Group').mean()
relief

In [ ]:
#: the observed group difference
reliefs = relief.get('Result')
observed_difference = reliefs.loc['Treatment'] - reliefs.loc['Control']
observed_difference

## The simulation

In [ ]:
np.random.seed(4242)
#: run a permutation test

n_repetitions = 500

differences = np.array([])

for i in np.arange(n_repetitions):
    # permute the results
    shuffled_results = np.random.permutation(back.get('Result'))
    
    shuffled = back.assign(ShuffledResult=shuffled_results)
    
    # compute the statistic
    relief = shuffled.groupby('Group').mean().get('ShuffledResult')
    difference = relief.loc['Treatment'] - relief.loc['Control']
    
    #append to array
    differences = np.append(differences, difference)
    


## Conclusion

In [ ]:
bpd.DataFrame().assign(DifferenceInMeans=differences).plot(kind='hist', bins=9, density=True)
plt.axvline(observed_difference, color='red')

In [ ]:
#p-value
np.count_nonzero(differences >= observed_difference) / n_repetitions

## Conclusion

- We reject the null hypothesis with a high degree of confidence.
- This is evidence that the treatment *caused* improvement.
    - *Because* this was an *randomized controlled trial*.

# Bootstrapping

## Question

- What is the median salary of San Diego city employees?
- All city employee salary data [is public](https://publicpay.ca.gov/Reports/Cities/City.aspx?entityid=405&year=2019&rpt=1).

In [ ]:
#: read in the data,
population = bpd.read_csv('data/2019_salaries.csv')
population

In [ ]:
population.columns

## Only need the total pay...

In [ ]:
population = population.get(['TotalWages'])
population

In [ ]:
population.plot(kind='hist')

## The median salary

- We can use `.median()`:

In [ ]:
#...population_median
population_median = population.get('TotalWages').median()
population_median

## But now...

- ...suppose we don't have access to this data.
- It is costly and time-consuming to survey *all* 12,000+ employees.
- So we gather salaries for a random sample of, say, 500 people.
- Hope the median of the sample $\approx$ median of the population.

## In the language of statistics...

- The full table of salaries is the **population**.
- We observe a **sample** of 500 salaries from the population.
- We really want the population median, but we don't have the whole population.
- So we compute sample median as an **estimate**.
- Hopefully the sample median is close to the population median.

## The sample median

- Let's survey 500 employees at random.
- We can use `.sample()`:

In [ ]:
#: take a sample of size 500
my_sample = population.sample(500, replace=False)

In [ ]:
#: compute the sample median
sample_median = my_sample.get('TotalWages').median()
sample_median

## How confident are we?

- Our estimate depended on a random sample.
- If our sample was different, our estimate would've been different, too.
- **How different could the estimate have been?**
- Our confidence in the estimate depends on the answer.

## The sample median is random

- The sample median is a random number.
- It comes from some distribution, which we don't know.
- How different could the estimate have been?
    - "Narrow" distribution $\Rightarrow$ not too different
    - "Wide" distribution $\Rightarrow$ quite different
- **What is the distribution of the sample median?**

## A (costly) approach

- Every sample of 500 people gives me one observation of the sample median.
- So draw a bunch of samples, compute medians.

In [ ]:
#: imports for animation
from lecture import sampling_animation
from IPython.display import HTML

In [ ]:
%%capture
anim, sample_medians = sampling_animation(population)

In [ ]:
#: display animation
HTML(anim.to_jshtml())

## Visualize the distribution

- We can plot the distribution of the sample median with a histogram.
- This is an approximation using 128 samples.
- Sample median is usually in [62,000, 72,000].

In [ ]:
#: plot a histogram
bpd.DataFrame().assign(SampleMedians=sample_medians).plot(kind='hist', bins=15, density=True)

## The problem

- Drawing new samples like this is costly (why not just do a census?)
- Often, we can't ask for new samples from the population.
- What about sampling the sample?
- The **bootstrap**:
    - the sample itself looks like the distribution.
    - so re-sampling from the sample is like drawing from the distribution.

In [ ]:
fig, ax = plt.subplots()
bins=np.arange(10_000, 300_000, 10_000)
population.plot(kind='hist', y='TotalWages', ax=ax, density=True, alpha=.75, bins=bins)
my_sample.plot(kind='hist', y='TotalWages', ax=ax, density=True, alpha=.75, bins=bins)
plt.legend(['Population', 'My Sample'])

## The bootstrap

- We have a sample of 500 salaries, we want another.
- Can't draw from the population.
- But the original sample looks like the population.
- So we re-sample the sample.

## Discussion question

Which of these effectively resamples the sample, simulating the drawing of a new sample of 500 people?

- A) `np.random.choice(my_sample, 500, replace=True)`
- B) `np.random.choice(my_sample, 500, replace=False)`
- C) `my_sample.sample(500, replace=True)`
- D) `my_sample.sample(500, replace=False)`

## Answer

- If we sample without replacement, we're just shuffling.
- So we sample *with* replacement to get something new.

## Running the bootstrap

- Now we can simulate new samples by bootstrapping
- I.e., we sample with replacement from our original sample

In [ ]:
n_resamples = 5000

boot_medians = np.array([])
for i in range(n_resamples):
    # perform bootstrap resampling
    resample = my_sample.sample(500, replace=True)
    
    # compute the median
    median = resample.get('TotalWages').median()
    
    # tack it on to our list of medians
    boot_medians = np.append(boot_medians, median)

## Bootstrap distribution of the sample median

- Bulk of the time, the sample median is typically in [62,000, 72,000].
- Similar to what we found before.
- The population median (red dot) is near the middle.

In [ ]:
#: visualize
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist')
plt.scatter(population_median, 0, color='red', s=80).set_zorder(2)

## Bootstrap rules of thumb

- The bootstrap is an awesome tool:
    - We used just one sample to get the (approximate) distribution of the sample median.
- But it has limitations:
    - Not good for sensitive statistics, like maximum.
    - Requires sample to be good approximation of population.
    - Can be slow (recommend 10,000+ bootstrap samples).

## Example: boostrapping in the German tank problem

- We observe a random sample of 30 tanks.
- Our goal: estimate total # of tanks from serial numbers of 30 tanks.

In [ ]:
#: we don't know this, but there are actually 400 planes in total
tank_population = bpd.DataFrame().assign(SerialNumber=np.arange(400)+1)

In [ ]:
#: sample 30 tanks
np.random.seed(4242)
tank_sample = tank_population.sample(30, replace=False)

## Running the bootstrap

- We want to estimate the maximum number in the population
- Our estimator will be the max in the sample.
- We run the bootstrap:

In [ ]:
n_resamples = 5000

boot_maxes = np.array([])
for i in range(n_resamples):
    # resample
    resample = tank_sample.sample(tank_sample.shape[0], replace=True)
    
    # compute max
    boot_max = resample.get('SerialNumber').max()
    
    boot_maxes = np.append(boot_maxes, boot_max)

## Visualize

- The bootstrap distribution doesn't surround the right maximum of 400. Why?

In [ ]:
bpd.DataFrame().assign(BootstrapMax=boot_maxes).plot(kind='hist', bins=20)
plt.scatter(400, 0, color='red')

In [ ]:
tank_sample.get('SerialNumber').max()

# Confidence Intervals

## Confidence intervals

- Bootstrapping approximates the distribution of an estimate
- The true value typically lies within bulk of the distribution
- Rather than returning only a single number, we can give an interval that we are confident that the correct value lies within

## A 95% confidence interval for median salary

- Recall our bootstrap distribution of the sample median
- Suppose by "bulk", we mean containing the middle 95% of the area.

In [ ]:
#: visualize
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist')

## Finding the endpoints

- We want to find two points, $x$ and $y$, such that the area:
    - to the left of $x$ is about 2.5%
    - to the right of $y$ is about 2.5%
- Then the interval $[x,y]$ will have about 95% of the total area
- I.e., we want the 2.5th **percentile** and 97.5th **percentile**.

## Computing percentiles

- Use `np.percentile(array, percentile)` function:
    - First arg: array of values
    - Second arg: percentile to find as # in [0, 100]

In [ ]:
# left
left = np.percentile(boot_medians, 2.5)
left

In [ ]:
# right
right = np.percentile(boot_medians, 97.5)
right

In [ ]:
#: our interval is
[left, right]

## Visualizing our 95% confidence interval

- Let's draw the interval on the histogram.
- 95% of the bootstrap medians fell into this interval.

In [ ]:
#: visualize
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist')
plt.plot([left, right], [0, 0], color='lime', linewidth=10)

## Discussion question

Would an 80% confidence interval be bigger, smaller, or the same size?

- A) Bigger
- B) Smaller
- C) The same size

## Discussion Question

Suppose you had the true distribution of the sample median and used it to compute a 100% confidence interval. And suppose you compute a 100% confidence interval using the bootstrap. Which is bigger?

- A) The first confidence interval (from the true distribution).
- B) The second confidence interval (from the boostrap).
- C) They're the same size.

## Interpreting confidence intervals

- 95% of our bootstrap medians fell within this interval
- We're pretty confident that the true median does, too.
- How confident should we be about this?

## Capturing the correct value

- If we run the bootstrap again, we get a different distribution.
- And so we get a different 95% confidence interval.
- (Roughly) 95% of the time, the interval will capture the correct median.

## Interpreting confidence intervals

- Doesn't have to be for same experiment!
- Suppose you only ever make 95% confidence intervals.
- Then roughly 95% of the CIs you make in your life will contain the true value of the thing being estimated.

## Misinterpreting confidence intervals

- A 95% confidence interval has a 95% chance of containing the true value of the thing being estimated.
- The interval is random, not the thing being estimated!

## Misinterpreting confidence intervals

- Our 95% confidence interval for the median salary was:

In [ ]:
#: remember...
[left, right]

- This does not mean that 95% of salaries are in this range!

In [ ]:
population.plot(kind='hist')
plt.plot([left, right], [0, 0], color='lime', linewidth=10)